In [1]:
import torch
import torch.nn as nn
import os
from transformers import BertTokenizer
from dataset import CustomDataset
from torch_geometric.loader import DataLoader

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
# 加载数据集
from datasets import load_dataset, DatasetDict

dataset_train = load_dataset('csv', data_files='data/train.csv', encoding='utf-8')
dataset_test = load_dataset('csv', data_files='data/test.csv', encoding='utf-8')
dataset_valid = load_dataset('csv', data_files='data/valid.csv', encoding='utf-8')
dataset = DatasetDict({'train': dataset_train, 'test': dataset_test, 'validation': dataset_valid})

In [3]:
# bert分词
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def encode_batch(batch):
    return bert_tokenizer(batch['text'], padding='max_length', truncation=True, max_length=72, return_tensors="pt")

dataset = {split: dataset[split].map(encode_batch, batched=True) for split in dataset.keys()} 
dataset['test']['train'] = dataset['test']['train'].remove_columns('text')
dataset['train']['train'] = dataset['train']['train'].remove_columns('text')
dataset['validation']['train'] = dataset['validation']['train'].remove_columns('text')

In [4]:
from utils import get_sentence_rel

# 获取边关系
train_sen_rel = get_sentence_rel(path='data/train.csv')
test_sen_rel = get_sentence_rel(path='data/test.csv')
valid_sen_rel = get_sentence_rel(path='data/valid.csv')


In [5]:
from utils import spilt_node

train_data = spilt_node(dataset['train']['train'])
test_data = spilt_node(dataset['test']['train'])

train_dataset = CustomDataset(train_data, train_sen_rel, 20, 8)
test_dataset = CustomDataset(test_data, test_sen_rel, 20, 8)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [6]:
# bert
from BertGAT import BertGAT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = RobertaGAT('roberta-base', num_classes=4)
model = BertGAT("bert-base-uncased", num_classes=5)
model.bert_first_half.to('cuda:0')
model.bert_second_half.to('cuda:1')

weights = torch.tensor([0.0855, 0.1915, 0.2766, 0.3404, 0.1915], dtype=torch.float32)
weights = weights.to('cuda:1')  
# weights = torch.tensor([0.1915, 0.2766, 0.3404, 0.1915], dtype=torch.float32)
# weights = weights.to('cuda:1')  

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)

criterion = nn.CrossEntropyLoss(weight=weights)
# criterion = nn.CrossEntropyLoss(ignore_index=4, weight=weights)
num_epochs = 20

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch[0]['x']
        attention_mask = batch[0]['mask']
        labels = batch[0]['y']
        edge_index = batch[0]['edge_index']
        num_nodes_graph = 0

        for i in range(1, len(batch)):
            edge_index_tmp = (batch[i]['edge_index'] + num_nodes_graph)
            edge_index = torch.cat((edge_index, edge_index_tmp), dim=1)
            num_nodes_graph += batch[i]['x'].size(0)
            input_ids = torch.cat((input_ids, batch[i]['x']), dim=0)
            attention_mask  = torch.cat((attention_mask, batch[i]['mask']), dim=0)
            labels = torch.cat((labels, batch[i]['y']), dim=0)

        input_ids = input_ids.to('cuda:0')
        attention_mask = attention_mask.to('cuda:0')
        edge_index = edge_index.to('cuda:1')
        labels = labels.to('cuda:1')
    
        optimizer.zero_grad()
        output, weight1 = model(input_ids, attention_mask, edge_index)
        loss = criterion(output.to("cuda:1"), labels)
        # print(loss)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        torch.cuda.empty_cache()

    avg_loss = total_loss / len(train_loader)
    
    torch.save(model.state_dict(), f'./model/model_epoch_{epoch}.pth')
    model.eval()  # 测试
    total_test_loss = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():  
        for batch in test_loader:
            input_ids = batch[0]['x']
            attention_mask = batch[0]['mask']
            labels = batch[0]['y']
            edge_index = batch[0]['edge_index']
            num_nodes_graph = 0
            
            for i in range(1, len(batch)):
                edge_index_tmp = (batch[i]['edge_index'] + num_nodes_graph)
                edge_index = torch.cat((edge_index, edge_index_tmp), dim=1)
                num_nodes_graph += batch[i]['x'].size(0)
                input_ids = torch.cat((input_ids, batch[i]['x']), dim=0)
                attention_mask  = torch.cat((attention_mask, batch[i]['mask']), dim=0)
                labels = torch.cat((labels, batch[i]['y']), dim=0)
                
            input_ids = input_ids.to('cuda:0')
            attention_mask = attention_mask.to('cuda:0')
            edge_index = edge_index.to('cuda:1')
            labels = labels.to('cuda:1')
            
            output, weight1 = model(input_ids, attention_mask, edge_index)
            loss = criterion(output.to("cuda:1"), labels)
            total_test_loss += loss.item()
            
            preds = output.argmax(dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            torch.cuda.empty_cache()
            
    avg_test_loss = total_test_loss / len(test_loader)

    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss}, Testing Loss: {avg_test_loss}, Accuracy: {accuracy}, Recall: {recall}, F1 Score: {f1}") 

Epoch 1/20, Training Loss: 0.6359631828554384, Testing Loss: 0.4623970452107881, Accuracy: 0.8250085333940153, Recall: 0.826934546874357, F1 Score: 0.8260683227623986
Epoch 2/20, Training Loss: 0.4225195901734488, Testing Loss: 0.4072001615637227, Accuracy: 0.8517465013084538, Recall: 0.8541573795735575, F1 Score: 0.8544352639129503
